In [ ]:
!pip -q install --no-cache-dir \
  "numpy==2.0.2" \
  "pandas==2.2.2" \
  "torch" \
  "transformers==4.48.3" \
  "sentencepiece==0.2.0" \
  "tqdm==4.66.5"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 179.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 128.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 254.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 358.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 133.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 1.0.1 requires tqdm>=4.67, but you have tqdm 4.66.5 which is incompatible.
tobler 0.13.0 requires tqdm>=4.67, but you have tqdm 4.66.5 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

import pandas as pd

ART_DIR = "/content/drive/MyDrive/uci_rag_chunks" #change folder name
chunks_df = pd.read_parquet(f"{ART_DIR}/chunks.parquet")

print("Chunks:", len(chunks_df))
chunks_df.head(2)

Mounted at /content/drive
Chunks: 213133


,review_id,chunk_index,drug,condition,rating,date,usefulcount,text,chunk_id
0,0,0,Valsartan,Left Ventricular Dysfunction,9.0,"May 20, 2012",27,"""It has no side effect, I take it in combinati...",0
1,1,0,Guanfacine,ADHD,8.0,"April 27, 2010",192,"""My son is halfway through his fourth week of ...",1


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

ENCODER_ID = "sentence-transformers/all-MiniLM-L6-v2"

tokenizer = AutoTokenizer.from_pretrained(ENCODER_ID)
model = AutoModel.from_pretrained(ENCODER_ID).to(device)
model.eval()

Device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 384, padding_idx=0)
    (position_embeddings): Embedding(512, 384)
    (token_type_embeddings): Embedding(2, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-5): 6 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
@torch.no_grad()
def mean_pool(last_hidden_state, attention_mask):
    mask = attention_mask.unsqueeze(-1)
    summed = (last_hidden_state * mask).sum(dim=1)
    counts = mask.sum(dim=1).clamp(min=1e-9)
    return summed / counts

In [ ]:
from tqdm import tqdm

texts = chunks_df["text"].tolist()

def embed_texts(texts, batch_size=128, max_length=256):
    all_embs = []

    for i in tqdm(range(0, len(texts), batch_size), desc="Embedding corpus"):
        batch = texts[i:i+batch_size]

        inputs = tokenizer(
            batch,
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        ).to(device)

        outputs = model(**inputs)
        pooled = mean_pool(outputs.last_hidden_state, inputs["attention_mask"])

        # Normalize → cosine similarity via dot product
        pooled = torch.nn.functional.normalize(pooled, p=2, dim=1)

        all_embs.append(pooled.cpu())

    return torch.cat(all_embs, dim=0)

dense_matrix = embed_texts(texts, batch_size=128 if device=="cuda" else 32)

print("Dense matrix:", dense_matrix.shape)

Embedding corpus: 100%|██████████| 1666/1666 [02:08<00:00, 13.00it/s]


Dense matrix: torch.Size([213133, 384])


In [ ]:
torch.save(dense_matrix, f"{ART_DIR}/dense_matrix.pt")

In [ ]:
dense_matrix = torch.load(f"{ART_DIR}/dense_matrix.pt")

In [ ]:
@torch.no_grad()
def embed_query(query, max_length=256):
    inputs = tokenizer(
        [query],
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    ).to(device)

    outputs = model(**inputs)
    pooled = mean_pool(outputs.last_hidden_state, inputs["attention_mask"])
    pooled = torch.nn.functional.normalize(pooled, p=2, dim=1)

    return pooled.cpu()  # shape [1, D]

In [ ]:
def dense_search(query, k=8):
    qv = embed_query(query)           # [1, D]
    scores = (dense_matrix @ qv.T)   # cosine since normalized
    scores = scores.squeeze(1)

    top = torch.topk(scores, k=k)

    hits = []
    for score, idx in zip(top.values, top.indices):
        row = chunks_df.iloc[int(idx)]
        hits.append({
            "score": float(score),
            "chunk_id": int(row["chunk_id"]),
            "drug": row.get("drug"),
            "condition": row.get("condition"),
            "text": row["text"]
        })
    return hits

hits = dense_search("dizziness nausea after starting medication", k=5)

for h in hits:
    print("\n---")
    print("score:", round(h["score"],4))
    print(h["text"][:400])


---
score: 0.7175
"Had bad stomach aches after taking the medication. Also dizziness and nausea. After a while the side effects lessened."

---
score: 0.7175
"Had bad stomach aches after taking the medication. Also dizziness and nausea. After a while the side effects lessened."

---
score: 0.7065
"I have taken 4 of these pills. The first one gave me no pain relief. I took a second 3 hours later. I got pain relief, also dizziness, exhaustion, inability to sleep, crying spells. I tried two pills at bedtime hoping to get good pain relief and be able to sleep. Was up all night, restless, tossing and turning. Woke up this morning and now 10 hours after taking the medicines am still nauseated an

---
score: 0.7065
"I have taken 4 of these pills. The first one gave me no pain relief. I took a second 3 hours later. I got pain relief, also dizziness, exhaustion, inability to sleep, crying spells. I tried two pills at bedtime hoping to get good pain relief and be able to sleep. Was up all night

In [ ]:
def hybrid_search(query, k=8, alpha=0.5):
    bm_hits = bm25_search(query, k=60)   # shortlist
    idxs = [h["chunk_id"] for h in bm_hits]

    qv = embed_query(query)
    cand = dense_matrix[idxs]

    dense_scores = (cand @ qv.T).squeeze(1)

    final = []
    for i, h in enumerate(bm_hits[:k]):
        final.append({
            **h,
            "dense_score": float(dense_scores[i])
        })
    return final

Save Dense Embeddings

In [ ]:
import os
OUT_DIR = "/content/drive/MyDrive/uci_rag_embeddings"  #change folder name for saving embeddings
os.makedirs(OUT_DIR, exist_ok=True)

torch.save(dense_matrix, f"{OUT_DIR}/dense_matrix.pt")

print("Saved:", f"{OUT_DIR}/dense_matrix.pt")

Saved: /content/drive/MyDrive/uci_rag_embeddings/dense_matrix.pt


In [ ]:
PT_PATH = "/content/drive/MyDrive/uci_rag_embeddings/dense_matrix.pt"

dense_matrix = torch.load(PT_PATH, map_location="cpu")

print("Loaded tensor shape:", dense_matrix.shape)
print("dtype:", dense_matrix.dtype)

Loaded tensor shape: torch.Size([213133, 384])
dtype: torch.float32


In [ ]:
import numpy as np

dense_np = dense_matrix.cpu().numpy().astype("float32") #save embeddings to numpy

print("Converted numpy shape:", dense_np.shape)
print("dtype:", dense_np.dtype)